In [1]:
!pip install pathway bokeh --quiet

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [3]:
# Install gdown to download files from Google Drive
!pip install gdown --quiet

# Extracted file ID from your Google Drive link
file_id = '1RqHF3zphAFOtYZgReDJUxEFweOiVAxqP'
output_path = 'your_dataset.csv'  # You can rename this

# Download the file from Google Drive
!gdown --id {file_id} -O {output_path} --quiet

# Load the CSV into a pandas DataFrame
import pandas as pd
df = pd.read_csv(output_path)

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(


In [4]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Make sure Timestamp is datetime type
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Sort first by Timestamp, then by SystemCodeNumber (parking location ID)
df = df.sort_values(by=['Timestamp', 'SystemCodeNumber']).reset_index(drop=True)

# Combine 'Latitude' and 'Longitude' into a single 'Location' column
df['Location'] = df['Latitude'].astype(str) + ' ' + df['Longitude'].astype(str)

In [5]:
print(len(df['SystemCodeNumber'].unique()))

print(df['Location'].unique())
print(df['TrafficConditionNearby'].unique())
print(df['VehicleType'].unique())

print(df['QueueLength'].unique())

14
['26.14453614 91.73617216' '26.14449459 91.73620513'
 '26.14901995 91.7395035' '20.0000347 78.00000286'
 '26.14001386 91.73099967' '26.14004753 91.73097233'
 '26.13795775 91.74099445' '26.14747299 91.72804914'
 '26.14754061 91.72797041' '26.14749943 91.72800489'
 '26.14749053 91.72799688' '26.14754886 91.72799519'
 '26.14749998 91.72797778' '26.15050395 91.73353109']
['low' 'average' 'high']
['car' 'bike' 'cycle' 'truck']
[ 1  2  3  5  4  6  7  9  8 10 11 13 12 14  0 15]


In [6]:
print(df["IsSpecialDay"].dtype)

int64


In [7]:
@pw.udf
def encode_location(loc: str) -> int:
    mapping = {
        '26.14453614 91.73617216': 1,
        '26.14449459 91.73620513': 2,
        '26.14901995 91.7395035': 3,
        '20.0000347 78.00000286': 4,
        '26.14001386 91.73099967': 5,
        '26.14004753 91.73097233': 6,
        '26.13795775 91.74099445': 7,
        '26.14747299 91.72804914': 8,
        '26.14754061 91.72797041': 9,
        '26.14749943 91.72800489': 10,
        '26.14749053 91.72799688': 11,
        '26.14754886 91.72799519': 12,
        '26.14749998 91.72797778': 13,
        '26.15050395 91.73353109': 14,
    }
    return mapping.get(loc, 0)

@pw.udf
def encode_vehicle_type(v) -> int:
    mapping = {'cycle': 1, 'bike': 2, 'car': 3, 'truck': 4}
    return mapping.get(v, 0)

@pw.udf
def encode_traffic(t) -> int:
    mapping = {'low': 1, 'average': 2, 'high': 3}
    return mapping.get(t, 0)

In [8]:
df_small = df[["Timestamp", "Occupancy", "Capacity", "Location", "QueueLength", "TrafficConditionNearby", "IsSpecialDay", "VehicleType"]]

df_small.to_csv("parking_stream_small.csv", index=False)

In [9]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    Location: str
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str

In [10]:
data = pw.demo.replay_csv("parking_stream_small.csv", schema=ParkingSchema, input_rate=1000)

data = data.with_columns(
    EncodedLocation = encode_location(data.Location),
    EncodedTraffic = encode_traffic(data.TrafficConditionNearby),
    EncodedVehicle = encode_vehicle_type(data.VehicleType),
)

data = data.select(
    data.Timestamp,
    data.Occupancy,
    data.Capacity,
    data.QueueLength,
    data.IsSpecialDay,
    data.EncodedLocation,
    data.EncodedTraffic,
    data.EncodedVehicle,
)

In [11]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    Time = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [12]:
demand = data_with_time.windowby(
    pw.this.Time,
    window=pw.temporal.tumbling(datetime.timedelta(minutes=30)),
    instance=pw.this.EncodedLocation,
    behavior=pw.temporal.exactly_once_behavior()
).reduce(
    Time = pw.this._pw_window_end,
    Location = pw.this._pw_instance,
    Occupancy_sum = pw.reducers.sum(pw.this.Occupancy),
    Capacity_sum = pw.reducers.sum(pw.this.Capacity),
    Queue_sum = pw.reducers.sum(pw.this.QueueLength),
    Traffic_sum = pw.reducers.sum(pw.this.EncodedTraffic),
    SpecialDay_sum = pw.reducers.sum(pw.this.IsSpecialDay),
    Vehicle_sum = pw.reducers.sum(pw.this.EncodedVehicle),
    Count = pw.reducers.count(),
).with_columns(
    Occupancy = pw.this.Occupancy_sum / pw.this.Count,
    Capacity = pw.this.Capacity_sum / pw.this.Count,
    QueueLength = pw.this.Queue_sum / pw.this.Count,
    TrafficCondition = pw.this.Traffic_sum / pw.this.Count,
    IsSpecialDay = pw.this.SpecialDay_sum / pw.this.Count,
    VehicleType = pw.this.Vehicle_sum / pw.this.Count,
).with_columns(
    NormalDemand = 5 * (pw.this.Occupancy / pw.this.Capacity)
                 + (3/20)*pw.this.QueueLength
                 + (1/6)*pw.this.TrafficCondition
                 + (1/2)*pw.this.IsSpecialDay
                 + (1/4)*pw.this.VehicleType
)

In [13]:
 result = demand.with_columns(
    Price = 10 * (1 + (1/10) * pw.this.NormalDemand)
).select(
    pw.this.Time,
    pw.this.Location,
    pw.this.Price
)

In [18]:
pw.debug.compute_and_print(result)

            | Time                | Location | Price
^YYYF4K0... | 2016-10-04 08:00:00 | 1        | 11.59526285384171
^Z3QGMSJ... | 2016-10-04 08:00:00 | 2        | 12.76705426356589
^3CZ2PQC... | 2016-10-04 08:00:00 | 3        | 12.461347517730497
^3HNA13W... | 2016-10-04 08:00:00 | 4        | 13.13806404657933
^3S2M9HE... | 2016-10-04 08:00:00 | 5        | 11.954166666666666
^A984K27... | 2016-10-04 08:00:00 | 6        | 13.783676975945017
^6A0GA4X... | 2016-10-04 08:00:00 | 7        | 12.506521739130436
^SN09EAA... | 2016-10-04 08:00:00 | 8        | 12.981226148996184
^9KM7YTE... | 2016-10-04 08:00:00 | 9        | 11.414508264954215
^QEC18B4... | 2016-10-04 08:00:00 | 10       | 12.608631642201047
^03PYNAM... | 2016-10-04 08:00:00 | 11       | 11.897889002383383
^GFDDF99... | 2016-10-04 08:00:00 | 12       | 12.966389309127585
^3J2QPDJ... | 2016-10-04 08:00:00 | 13       | 12.330803523471909
^XTPJ98H... | 2016-10-04 08:00:00 | 14       | 12.315624999999999
^YYYB3FZ... | 2016-10-04 0

In [19]:
import io
import sys
import pandas as pd

# Capture the printed output
buffer = io.StringIO()
sys.stdout = buffer

pw.debug.compute_and_print(result)  # triggers run
sys.stdout = sys.__stdout__         # reset stdout

# Parse the output
raw_output = buffer.getvalue().strip().split("\n")
rows = [line.split("|") for line in raw_output if "|" in line]
columns = [col.strip() for col in rows[0]]
data = [[val.strip() for val in row] for row in rows[1:]]

# Convert to DataFrame
df = pd.DataFrame(data, columns=columns)

# Fix data types
df["Time"] = pd.to_datetime(df["Time"])
df["Location"] = df["Location"].astype(int)
df["Price"] = df["Price"].astype(float)

In [22]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category20_20
from bokeh.layouts import column

output_notebook()

p = figure(x_axis_type="datetime", title="Price per Parking Space", width=1000, height=500)
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Price'

colors = Category20_20 * 2  # Up to 40 lines

for i, space in enumerate(sorted(df["Location"].unique())):
    df_space = df[df["Location"] == space]
    source = ColumnDataSource(df_space)
    p.line(x='Time', y='Price', source=source, legend_label=f"Space {space}", line_width=2, color=colors[i % len(colors)])

p.legend.title = "Parking Space"
p.legend.location = "top_left"
p.legend.label_text_font_size = "8pt"

p.add_tools(HoverTool(
    tooltips=[("Time", "@Time{%F %T}"), ("Price", "@Price"), ("Space", "@Location")],
    formatters={"@Time": "datetime"},
    mode='mouse'
))

show(p)